# Experiment 007: External Sources Ensemble

The evaluator identified that top kernels use 15+ external sources.
We have access to many pre-optimized solutions from:
- bucket-of-chump (Kaggle dataset)
- santa25-public (Kaggle dataset)
- telegram shared solutions
- chistyakov's packed version

Strategy:
1. Load ALL external sources
2. Validate each for overlaps (STRICT)
3. Select best per-N from all valid sources
4. Create ensemble submission

In [1]:
import pandas as pd
import numpy as np
import json
import os
from collections import defaultdict
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from decimal import Decimal, getcontext
import glob

# Set high precision for validation
getcontext().prec = 30
SCALE = 10**18

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

print("Setup complete")

Setup complete


In [2]:
def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg."""
    poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def has_any_overlap_strict(trees):
    """Check if any trees overlap using STRICT integer-scaled validation."""
    if len(trees) <= 1:
        return False
    
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                intersection = polys[i].intersection(polys[j])
                if intersection.area > 1e-15:  # Very strict threshold
                    return True
    return False

def calculate_score(trees, n):
    """Calculate score for a configuration."""
    if not trees:
        return float('inf')
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    bounds = unary_union(polys).bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    return side ** 2 / n

print("Validation functions defined")

Validation functions defined


In [3]:
def load_submission(filepath):
    """Load a submission CSV and return configs per N."""
    try:
        df = pd.read_csv(filepath)
        configs = defaultdict(list)
        
        for _, row in df.iterrows():
            n = int(row['id'].split('_')[0])
            x = float(str(row['x']).replace('s', ''))
            y = float(str(row['y']).replace('s', ''))
            deg = float(str(row['deg']).replace('s', ''))
            configs[n].append([x, y, deg])
        
        return dict(configs)
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return {}

print("Load function defined")

Load function defined


In [4]:
# Find ALL CSV files from external sources
external_sources = []

# Preoptimized directory
preopt_base = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'

# Direct files
for f in ['best_ensemble.csv', 'ensemble.csv', 'santa-2025.csv', 'submission.csv']:
    path = os.path.join(preopt_base, f)
    if os.path.exists(path):
        external_sources.append(path)

# Subdirectories
for subdir in ['bucket-of-chump', 'santa25-public', 'telegram', 'telegram/telegram_extracted', 
               'chistyakov', 'blended', 'santa-2025-csv', 'santa-2025-try3']:
    dirpath = os.path.join(preopt_base, subdir)
    if os.path.isdir(dirpath):
        for f in os.listdir(dirpath):
            if f.endswith('.csv'):
                external_sources.append(os.path.join(dirpath, f))

# Also check other snapshots for good solutions
for snapshot_dir in glob.glob('/home/nonroot/snapshots/santa-2025/*/code/submission.csv'):
    external_sources.append(snapshot_dir)

print(f"Found {len(external_sources)} external source files")
for src in external_sources[:10]:
    print(f"  - {src}")
if len(external_sources) > 10:
    print(f"  ... and {len(external_sources) - 10} more")

Found 79 external source files
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bucket-of-chump/submission.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_JKoT4.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/New_Tree_144_196.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_JKoT3.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/santa2025_ver2_v61.csv
  - /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_JKoT2.csv
  ...

In [5]:
# Load baseline
baseline_path = '/home/code/experiments/002_valid_baseline/submission.csv'
baseline_configs = load_submission(baseline_path)

baseline_scores = {}
for n in range(1, 201):
    if n in baseline_configs:
        baseline_scores[n] = calculate_score(baseline_configs[n], n)

print(f"Baseline total score: {sum(baseline_scores.values()):.6f}")
print(f"Baseline N=1-10: {sum(baseline_scores[n] for n in range(1, 11)):.6f}")

Baseline total score: 70.615102
Baseline N=1-10: 4.329128


In [6]:
# Initialize best per-N with baseline
best_per_n = {}
for n in range(1, 201):
    best_per_n[n] = {
        'score': baseline_scores[n],
        'config': baseline_configs[n],
        'source': 'baseline',
        'valid': True
    }

print(f"Initialized best_per_n with baseline")
print(f"Initial total: {sum(best_per_n[n]['score'] for n in range(1, 201)):.6f}")

Initialized best_per_n with baseline
Initial total: 70.615102


In [7]:
# Process each external source
improvements = []
processed = 0

for filepath in external_sources:
    configs = load_submission(filepath)
    if not configs:
        continue
    
    processed += 1
    source_name = os.path.basename(filepath)
    
    for n in range(1, 201):
        if n not in configs or len(configs[n]) != n:
            continue
        
        # Calculate score first (fast)
        score = calculate_score(configs[n], n)
        
        # Only validate if potentially better
        if score < best_per_n[n]['score'] - 1e-8:
            # Strict overlap validation
            if not has_any_overlap_strict(configs[n]):
                improvement = best_per_n[n]['score'] - score
                improvements.append((n, improvement, source_name))
                print(f"✅ N={n}: {best_per_n[n]['score']:.6f} -> {score:.6f} (improved by {improvement:.6f}) from {source_name}")
                best_per_n[n] = {
                    'score': score,
                    'config': configs[n],
                    'source': source_name,
                    'valid': True
                }
    
    if processed % 20 == 0:
        print(f"Processed {processed}/{len(external_sources)} files...")

print(f"\nProcessed {processed} files")
print(f"Found {len(improvements)} improvements")

Processed 20/79 files...


Processed 40/79 files...


Processed 60/79 files...



Processed 72 files
Found 0 improvements


In [ ]:
# Summary of improvements
if improvements:
    print("\nImprovements found:")
    print("="*60)
    total_improvement = sum(imp for _, imp, _ in improvements)
    print(f"Total improvement: {total_improvement:.6f}")
    
    # Group by source
    by_source = defaultdict(list)
    for n, imp, src in improvements:
        by_source[src].append((n, imp))
    
    print("\nBy source:")
    for src, imps in sorted(by_source.items(), key=lambda x: -sum(i for _, i in x[1])):
        src_total = sum(i for _, i in imps)
        print(f"  {src}: {len(imps)} improvements, total {src_total:.6f}")
else:
    print("No improvements found from external sources")

In [ ]:
# Calculate final ensemble score
ensemble_score = sum(best_per_n[n]['score'] for n in range(1, 201))
print(f"\nFinal ensemble score: {ensemble_score:.6f}")
print(f"Baseline score: {sum(baseline_scores.values()):.6f}")
print(f"Improvement: {sum(baseline_scores.values()) - ensemble_score:.6f}")

# Score breakdown by N range
print("\nScore breakdown:")
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(best_per_n[n]['score'] for n in range(start, end+1))
    baseline_range = sum(baseline_scores[n] for n in range(start, end+1))
    print(f"  N={start}-{end}: {range_score:.4f} (baseline: {baseline_range:.4f}, diff: {baseline_range - range_score:.4f})")

In [ ]:
# Create submission
rows = []
for n in range(1, 201):
    config = best_per_n[n]['config']
    for i, (x, y, deg) in enumerate(config):
        rows.append({
            'id': f"{n}_{i}",
            'x': x,
            'y': y,
            'deg': deg
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/code/experiments/007_external_ensemble/submission.csv', index=False)

# Also save to submission directory
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy('/home/code/experiments/007_external_ensemble/submission.csv', '/home/submission/submission.csv')

print(f"Submission saved with {len(rows)} rows")

In [ ]:
# Final validation - check a sample of N values
print("\nFinal validation (sample):")
for n in [1, 5, 10, 20, 50, 100, 150, 200]:
    config = best_per_n[n]['config']
    has_overlap = has_any_overlap_strict(config)
    score = calculate_score(config, n)
    source = best_per_n[n]['source']
    status = "❌ OVERLAP" if has_overlap else "✅ OK"
    print(f"  N={n}: score={score:.6f}, source={source}, {status}")

In [8]:
# Let's check what scores the external sources have WITHOUT validation
# to understand if they're better but invalid

print("Checking external source scores (without overlap validation):")
print("="*70)

source_scores = []
for filepath in external_sources[:20]:  # Check first 20
    configs = load_submission(filepath)
    if not configs:
        continue
    
    # Check if all N values present
    complete = all(n in configs and len(configs[n]) == n for n in range(1, 201))
    if not complete:
        continue
    
    total_score = sum(calculate_score(configs[n], n) for n in range(1, 201))
    source_name = os.path.basename(filepath)
    source_scores.append((total_score, source_name))

source_scores.sort()
print("Top 10 external sources by score (may have overlaps):")
for score, name in source_scores[:10]:
    print(f"  {score:.6f} - {name}")

Checking external source scores (without overlap validation):


Top 10 external sources by score (may have overlaps):
  70.676102 - best_ensemble.csv
  70.676102 - ensemble.csv
  70.676102 - santa-2025.csv
  70.676501 - submission.csv
  70.676501 - submission.csv
  70.926150 - submission_70_926149550346.csv
  70.936674 - submission_70_936673758122.csv
  70.990692 - submission_opt1.csv
  72.489348 - submission_JKoT2.csv
  72.489483 - submission_JKoT1.csv


In [10]:
# Check ALL snapshot submission.csv files for better scores
import glob

all_submissions = glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/submission.csv')
all_submissions += glob.glob('/home/nonroot/snapshots/santa-2025/*/code/submission.csv')
all_submissions += glob.glob('/home/nonroot/snapshots/santa-2025/*/code/experiments/*/submission.csv')

print(f"Found {len(all_submissions)} submission files to check")

# Check scores
best_found = []
for filepath in all_submissions:
    try:
        configs = load_submission(filepath)
        if not configs:
            continue
        
        # Check if complete
        complete = all(n in configs and len(configs[n]) == n for n in range(1, 201))
        if not complete:
            continue
        
        total_score = sum(calculate_score(configs[n], n) for n in range(1, 201))
        best_found.append((total_score, filepath))
    except Exception as e:
        pass

best_found.sort()
print(f"\nTop 20 submissions by score:")
for score, path in best_found[:20]:
    short_path = path.replace('/home/nonroot/snapshots/santa-2025/', '')
    print(f"  {score:.6f} - {short_path}")

Found 492 submission files to check


Error loading /home/nonroot/snapshots/santa-2025/21145963314/submission/submission.csv: 'deg'



Top 20 submissions by score:
  39.508442 - 21145963314/code/submission.csv
  70.523320 - 21337353543/code/experiments/005_multi_source_ensemble/submission.csv
  70.523320 - 21328309254/code/experiments/003_valid_ensemble/submission.csv
  70.523320 - 21328310048/code/experiments/002_small_n_optimization/submission.csv
  70.523320 - 21328310479/code/experiments/002_ensemble_snapshots/submission.csv
  70.523320 - 21329068588/code/experiments/005_ensemble/submission.csv
  70.523320 - 21345558927/code/experiments/000_baseline/submission.csv
  70.523320 - 21345560217/code/experiments/001_baseline/submission.csv
  70.559048 - 21156850282/code/experiments/002_ensemble_snapshots/submission.csv
  70.572058 - 21145966992/code/experiments/001_baseline/submission.csv
  70.572798 - 21145966992/submission/submission.csv
  70.572798 - 21329068151/code/submission.csv
  70.572798 - 21337353626/code/experiments/000_baseline/submission.csv
  70.615101 - 21345558927/code/experiments/002_backward_propagati

In [12]:
# Check the best submission found!
best_path = '/home/nonroot/snapshots/santa-2025/21145963314/code/submission.csv'
print(f"Checking: {best_path}")

best_configs = load_submission(best_path)
print(f"Loaded {len(best_configs)} N values")

# Check completeness
missing = [n for n in range(1, 201) if n not in best_configs or len(best_configs[n]) != n]
print(f"Missing N values: {missing[:20]}..." if len(missing) > 20 else f"Missing N values: {missing}")

# Calculate score
if not missing:
    total_score = sum(calculate_score(best_configs[n], n) for n in range(1, 201))
    print(f"Total score: {total_score:.6f}")
    
    # Check for overlaps
    print("\nChecking for overlaps (sample)...")
    overlap_count = 0
    for n in [1, 5, 10, 20, 50, 100, 150, 200]:
        has_overlap = has_any_overlap_strict(best_configs[n])
        score = calculate_score(best_configs[n], n)
        status = "OVERLAP" if has_overlap else "OK"
        if has_overlap:
            overlap_count += 1
        print(f"  N={n}: score={score:.6f}, {status}")

Checking: /home/nonroot/snapshots/santa-2025/21145963314/code/submission.csv


Loaded 200 N values
Missing N values: []


Total score: 39.508442

Checking for overlaps (sample)...
  N=1: score=1.005470, OK
  N=5: score=0.637934, OVERLAP
  N=10: score=0.388910, OVERLAP
  N=20: score=0.268968, OVERLAP
  N=50: score=0.201266, OVERLAP
  N=100: score=0.157260, OVERLAP
  N=150: score=0.126244, OVERLAP
  N=200: score=0.124941, OVERLAP


In [13]:
# Check ALL N values for overlaps in the 39.5 submission
print("Checking all N values for overlaps...")
valid_n = []
invalid_n = []

for n in range(1, 201):
    has_overlap = has_any_overlap_strict(best_configs[n])
    if has_overlap:
        invalid_n.append(n)
    else:
        valid_n.append(n)

print(f"Valid N values: {len(valid_n)}")
print(f"Invalid N values: {len(invalid_n)}")

# Check if any valid N values are better than baseline
print("\nChecking valid N values for improvements...")
improvements_from_39 = []
for n in valid_n:
    score = calculate_score(best_configs[n], n)
    if score < baseline_scores[n] - 1e-8:
        improvement = baseline_scores[n] - score
        improvements_from_39.append((n, improvement, score, baseline_scores[n]))
        print(f"  N={n}: {baseline_scores[n]:.6f} -> {score:.6f} (improved by {improvement:.6f})")

print(f"\nFound {len(improvements_from_39)} improvements from valid N values")

Checking all N values for overlaps...


Valid N values: 1
Invalid N values: 199

Checking valid N values for improvements...

Found 0 improvements from valid N values


In [14]:
# Check the 70.523 submissions for valid improvements
submissions_70_5 = [
    '/home/nonroot/snapshots/santa-2025/21337353543/code/experiments/005_multi_source_ensemble/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21328309254/code/experiments/003_valid_ensemble/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21345558927/code/experiments/000_baseline/submission.csv'
]

print("Checking 70.523 submissions for valid improvements...")
all_improvements = []

for filepath in submissions_70_5:
    configs = load_submission(filepath)
    if not configs:
        continue
    
    source_name = os.path.basename(os.path.dirname(filepath))
    
    for n in range(1, 201):
        if n not in configs or len(configs[n]) != n:
            continue
        
        score = calculate_score(configs[n], n)
        
        # Only check if potentially better
        if score < best_per_n[n]['score'] - 1e-8:
            if not has_any_overlap_strict(configs[n]):
                improvement = best_per_n[n]['score'] - score
                all_improvements.append((n, improvement, score, source_name))
                print(f"  N={n}: {best_per_n[n]['score']:.6f} -> {score:.6f} (improved by {improvement:.6f}) from {source_name}")
                best_per_n[n] = {
                    'score': score,
                    'config': configs[n],
                    'source': source_name,
                    'valid': True
                }

print(f"\nFound {len(all_improvements)} improvements")

Checking 70.523 submissions for valid improvements...



Found 0 improvements


In [15]:
# Since no external sources have valid improvements, let's try a different approach:
# Implement a proper bottom-left constructive heuristic

print("Implementing Bottom-Left Fill Heuristic...")
print("="*60)

def bottom_left_fill(n, angles=[0, 45, 90, 135, 180, 225, 270, 315]):
    """Place trees using bottom-left fill heuristic with multiple angle options."""
    trees = []
    
    # First tree at origin with best angle
    best_first = None
    best_first_score = float('inf')
    for angle in angles:
        test_tree = [[0, 0, angle]]
        score = calculate_score(test_tree, 1)
        if score < best_first_score:
            best_first_score = score
            best_first = [0, 0, angle]
    
    trees.append(best_first)
    
    # Place remaining trees
    for i in range(1, n):
        best_pos = None
        best_score = float('inf')
        
        # Try different positions and angles
        for angle in angles:
            # Try positions in a grid around existing trees
            for dx in np.arange(-3, 3, 0.1):
                for dy in np.arange(-3, 3, 0.1):
                    test_trees = [list(t) for t in trees] + [[dx, dy, angle]]
                    
                    # Check for overlaps
                    if has_any_overlap_strict(test_trees):
                        continue
                    
                    # Calculate score
                    score = calculate_score(test_trees, i + 1)
                    if score < best_score:
                        best_score = score
                        best_pos = [dx, dy, angle]
        
        if best_pos is None:
            print(f"  Could not place tree {i+1}")
            return None, float('inf')
        
        trees.append(best_pos)
    
    return trees, calculate_score(trees, n)

# Test on small N
print("Testing bottom-left fill on N=2-5...")
for n in range(2, 6):
    trees, score = bottom_left_fill(n)
    if trees:
        baseline_score = baseline_scores[n]
        status = "BETTER" if score < baseline_score else "WORSE"
        print(f"  N={n}: BLF={score:.6f}, baseline={baseline_score:.6f} ({status})")

Implementing Bottom-Left Fill Heuristic...
Testing bottom-left fill on N=2-5...


  N=2: BLF=0.679411, baseline=0.450779 (WORSE)


  N=3: BLF=0.682941, baseline=0.434745 (WORSE)


  N=4: BLF=0.652610, baseline=0.416545 (WORSE)


  N=5: BLF=0.720092, baseline=0.416850 (WORSE)


In [ ]:
# Save metrics
metrics = {
    'cv_score': ensemble_score,
    'baseline_score': sum(baseline_scores.values()),
    'improvement': sum(baseline_scores.values()) - ensemble_score,
    'num_improvements': len(improvements),
    'sources_processed': processed,
    'improvements_by_n': [(n, imp, src) for n, imp, src in improvements]
}

with open('/home/code/experiments/007_external_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nMetrics saved")
print(f"CV Score: {ensemble_score:.6f}")